In [1]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 163 kB 54.7 MB/s 
     |████████████████████████████████| 7.6 MB 40.6 MB/s 


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

Encode and Calculate Sentiment

In [6]:
tokens = tokenizer.encode('I hated this, absolutely the worst', return_tensors='pt')

In [7]:
tokens[0]

tensor([  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
          102])

In [8]:
#just to show how to decode
tokenizer.decode(tokens[0])

'[CLS] i hated this, absolutely the worst [SEP]'

In [9]:
result = model(tokens)

In [10]:
result.logits

tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>)

In [12]:
int(torch.argmax(result.logits))+1

1

Collect Reviews

In [13]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]

Load Reviews into DataFrame and Score

In [14]:
import pandas as pd
import numpy as np

In [17]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [18]:
df.head()

,review
0,"Great atmosphere, attentive service, solid mar..."
1,Don't come here expecting legit Mexican food b...
2,Out of all the restaurants that I tried in Syd...
3,The food is fresh and tasty. The scallop cevi...
4,We came here on a Thursday night @ 5pm and by ...


In [27]:
def sentiment_score(review):
  tokens = tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [28]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [29]:
df

,review,sentiment
0,"Great atmosphere, attentive service, solid mar...",3
1,Don't come here expecting legit Mexican food b...,3
2,Out of all the restaurants that I tried in Syd...,5
3,The food is fresh and tasty. The scallop cevi...,4
4,We came here on a Thursday night @ 5pm and by ...,4
5,Have been here twice and have absolutely loved...,5
6,I was pleasantly surprised at what a great job...,5
7,If you're looking for a quiet little romantic ...,2
8,The service at this place was top notch - the ...,5
9,Really nice (upmarket) Mexican restaurant. Goo...,4
